## **Imports**

In [39]:
# Imports
import pandas as pd  # Data manipulation and analysis
import numpy as np   # Support for large, multi-dimensional arrays and matrices
import os            # Interface with the operating system
%pip install lxml    # Install lxml library for XML and HTML processing
from lxml import etree  # XML and HTML manipulation
%pip install requests # Install requests library for HTTP operations
import requests      # Library to make HTTP requests
import xml.etree.ElementTree as ET  # XML parsing and manipulation
from collections import defaultdict  # Dictionary that provides a default value for missing keys


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## **Load and inspect data**

In [40]:
# Load from csv
person = pd.read_csv('data/Person Person.csv', delimiter=',')

# Inspect columns
person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19972 entries, 0 to 19971
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   BusinessEntityID       19972 non-null  int64 
 1   PersonType             19972 non-null  object
 2   NameStyle              19972 non-null  bool  
 3   Title                  1009 non-null   object
 4   FirstName              19972 non-null  object
 5   MiddleName             11473 non-null  object
 6   LastName               19972 non-null  object
 7   Suffix                 53 non-null     object
 8   EmailPromotion         19972 non-null  int64 
 9   AdditionalContactInfo  10 non-null     object
 10  Demographics           19972 non-null  object
 11  rowguid                19972 non-null  object
 12  ModifiedDate           19972 non-null  object
dtypes: bool(1), int64(2), object(10)
memory usage: 1.8+ MB


In [41]:
# Inspect data
person.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1722,IN,False,NaN,Mandar,NaN,Samant,NaN,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",6F123B92-FF1E-4F18-A054-F1477D965B36,2013-04-08 00:00:00.000
1,2389,IN,False,NaN,Laura,NaN,Liu,NaN,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",0BB7741C-53C5-4502-AABC-AA3E798EC6E7,2013-09-04 00:00:00.000
2,2391,IN,False,NaN,Laura,NaN,Wu,NaN,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",D2535AEE-7780-4CAC-9737-5D23895FD5B5,2012-04-20 00:00:00.000
3,2397,IN,False,NaN,Isabella,NaN,Cox,NaN,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",09FB996B-2BD4-454F-A9D2-F0F307D13039,2013-10-06 00:00:00.000
4,2401,IN,False,NaN,Laura,NaN,Gao,NaN,0,NaN,"<IndividualSurvey xmlns=""http://schemas.micros...",D319CB69-2139-4F0C-AEA6-541B3D5B0412,2014-05-15 00:00:00.000



1. There are a number of columns with high proportions of missing values: `Title`, `Suffix`, `AdditionalContactInfo`. Based on the column names, it's unlikely these will be useful for segmentation so we can remove them. 
2. Columns recording name information (`Firstname`, `MiddleName`, `LastName`) won't be needed for this project either, so we can remove these too. 
3. The `PersonType` column contains descriptive labels about each person's relationship to AdventureWorks. For initial analysis, we'll concentrate on "individual customers" (those with an "IN" value) so we can subset these individuals. 
4. The `Demographics` column contains XML values with demographic information that will be useful for segmentation. In order to work with the information, we'll need to extract it from XML format into structured columns.  

## **Subset and prepare data**

In [42]:
# Drop columns
person = person.drop(['NameStyle',
                      'Title',
                      'FirstName',
                      'MiddleName',
                      'LastName',
                      'Suffix',
                      'AdditionalContactInfo'], axis = 1)

# Subset individual customers
person_in = person[person['PersonType']=='IN']

# Drop PersonType column
person_in = person_in.drop('PersonType', axis = 1)

# Reset row indices to allow for concatenation later
person_in.reset_index(inplace=True, drop=True)

# Inspect data
person_in.head()


,BusinessEntityID,EmailPromotion,Demographics,rowguid,ModifiedDate
0,1722,0,"<IndividualSurvey xmlns=""http://schemas.micros...",6F123B92-FF1E-4F18-A054-F1477D965B36,2013-04-08 00:00:00.000
1,2389,0,"<IndividualSurvey xmlns=""http://schemas.micros...",0BB7741C-53C5-4502-AABC-AA3E798EC6E7,2013-09-04 00:00:00.000
2,2391,0,"<IndividualSurvey xmlns=""http://schemas.micros...",D2535AEE-7780-4CAC-9737-5D23895FD5B5,2012-04-20 00:00:00.000
3,2397,0,"<IndividualSurvey xmlns=""http://schemas.micros...",09FB996B-2BD4-454F-A9D2-F0F307D13039,2013-10-06 00:00:00.000
4,2401,0,"<IndividualSurvey xmlns=""http://schemas.micros...",D319CB69-2139-4F0C-AEA6-541B3D5B0412,2014-05-15 00:00:00.000


### Working with XML

XML is a flexible format for storing data in a tree-like structure, using nested tags and attributes. 

We want to parse the XML strings to extract the structured data into new columns. The function below is designed to do that. 

In [43]:
# Function to parse XML and extract all items
def xml_to_columns(xml_string):
    # Namespace for schema
    ns = {'aw': 'http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/IndividualSurvey'}
    
    # Parse the XML string into a tree structure
    tree = ET.fromstring(xml_string)
    
    # Initialize a dictionary to store items, defaulting missing entries to None
    items = defaultdict(lambda: None)
    
    # Iterate through each element in the XML tree, using the namespace map
    for element in tree.findall('.//aw:*', namespaces=ns):
        # Extract the tag and remove the namespace information for clarity
        tag = element.tag.split('}')[-1]  
        # Store the element's text content under the tag in the items dictionary
        items[tag] = element.text
    
    return pd.Series(items)

In [44]:
# Apply the function 
extracted_xml_columns = person_in['Demographics'].apply(xml_to_columns)

# Concatenate the new columns back onto the original dataframe
person_in = person_in.join(extracted_xml_columns)

# Drop the original Demographics columns, which is no longer needed
person_in = person_in.drop(['Demographics'], axis = 1)

# Inspect
person_in

,BusinessEntityID,EmailPromotion,rowguid,ModifiedDate,TotalPurchaseYTD,DateFirstPurchase,BirthDate,MaritalStatus,YearlyIncome,Gender,TotalChildren,NumberChildrenAtHome,Education,Occupation,HomeOwnerFlag,NumberCarsOwned,CommuteDistance
0,1722,0,6F123B92-FF1E-4F18-A054-F1477D965B36,2013-04-08 00:00:00.000,3499.8504,2003-05-10Z,1974-09-18Z,S,0-25000,F,0,0,Partial High School,Manual,0,2,0-1 Miles
1,2389,0,0BB7741C-53C5-4502-AABC-AA3E798EC6E7,2013-09-04 00:00:00.000,1085.5,2003-10-06Z,1967-10-05Z,S,25001-50000,F,5,5,High School,Skilled Manual,1,3,10+ Miles
2,2391,0,D2535AEE-7780-4CAC-9737-5D23895FD5B5,2012-04-20 00:00:00.000,36.59,2002-05-22Z,1973-10-21Z,S,greater than 100000,F,0,5,Partial College,Management,1,3,10+ Miles
3,2397,0,09FB996B-2BD4-454F-A9D2-F0F307D13039,2013-10-06 00:00:00.000,-3.99,2003-11-07Z,1978-08-28Z,S,25001-50000,F,0,0,Partial High School,Clerical,1,2,5-10 Miles
4,2401,0,D319CB69-2139-4F0C-AEA6-541B3D5B0412,2014-05-15 00:00:00.000,12.99,2004-06-15Z,1939-11-03Z,M,25001-50000,F,2,0,Bachelors,Management,1,2,5-10 Miles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,20737,0,CBD497C2-0C8E-4802-8414-E0C3715CB469,2014-03-04 00:00:00.000,719.5,2004-04-04Z,1973-03-28Z,M,25001-50000,F,1,1,Bachelors,Skilled Manual,1,1,1-2 Miles
18480,20751,0,5DD45523-9CB6-49B8-9FB4-80D454711B45,2013-02-06 00:00:00.000,-688.37,2003-03-10Z,1970-11-09Z,S,greater than 100000,F,0,5,Partial High School,Professional,0,4,10+ Miles
18481,1725,0,C3D0FAA0-190B-480F-9E38-CBAF6C9C5619,2013-12-26 00:00:00.000,-13.01,2004-01-27Z,1947-05-01Z,M,50001-75000,M,5,0,Bachelors,Management,1,3,10+ Miles
18482,1753,0,CF932CF1-F379-4386-A509-28E2EE9086BF,2014-03-20 00:00:00.000,-9.51,2004-04-20Z,1964-08-25Z,M,50001-75000,M,3,3,Bachelors,Professional,0,2,0-1 Miles


## **Pre-process and transform data**

After extraction from XML, values will be treated as strings by default. Therefore some processing of these fields will be required to make them suitable for Exploratory Data Analysis and statistical modelling. 


In [45]:
# Inspect columns
person_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   BusinessEntityID      18484 non-null  int64 
 1   EmailPromotion        18484 non-null  int64 
 2   rowguid               18484 non-null  object
 3   ModifiedDate          18484 non-null  object
 4   TotalPurchaseYTD      18484 non-null  object
 5   DateFirstPurchase     18484 non-null  object
 6   BirthDate             18484 non-null  object
 7   MaritalStatus         18484 non-null  object
 8   YearlyIncome          18484 non-null  object
 9   Gender                18484 non-null  object
 10  TotalChildren         18484 non-null  object
 11  NumberChildrenAtHome  18484 non-null  object
 12  Education             18484 non-null  object
 13  Occupation            18484 non-null  object
 14  HomeOwnerFlag         18484 non-null  object
 15  NumberCarsOwned       18484 non-null

### Data type transformations

As expected, all the newly extracted fields are `object` type. After checking the values, these are the data type transformations needed to make them suitable for modelling. 

1. Floats
    - `TotalPurchaseYTD`

2. Dates
    - `DateFirstPurchase`	
    - `BirthDate`
    - `ModifiedDate`

3. Categories
    - `MaritalStatus`	
    - `YearlyIncome`	
    - `Gender`
    - `Education`	
    - `Occupation`
    - `CommuteDistance`

4. Integers
    - `TotalChildren`	
    - `NumberChildrenAtHome`
    - `HomeOwnerFlag`	
    - `NumberCarsOwned`

In [46]:
# Float conversions
person_in['TotalPurchaseYTD'] = person_in['TotalPurchaseYTD'].astype(float)

# Datetime conversions
person_in['DateFirstPurchase'] = pd.to_datetime(person_in['DateFirstPurchase'], format='%Y-%m-%dZ')
person_in['BirthDate'] = pd.to_datetime(person_in['BirthDate'], format='%Y-%m-%dZ')
person_in['ModifiedDate'] = pd.to_datetime(person_in['ModifiedDate'])

# Category conversions
person_in['MaritalStatus'] = person_in['MaritalStatus'].astype('category')
person_in['YearlyIncome'] = person_in['YearlyIncome'].astype('category')
person_in['Gender'] = person_in['Gender'].astype('category')
person_in['Education'] = person_in['Education'].astype('category')
person_in['Occupation'] = person_in['Occupation'].astype('category')
person_in['CommuteDistance'] = person_in['CommuteDistance'].astype('category')

# Integer conversions
person_in['TotalChildren'] = person_in['TotalChildren'].astype(int)
person_in['NumberChildrenAtHome'] = person_in['NumberChildrenAtHome'].astype(int)
person_in['HomeOwnerFlag'] = person_in['HomeOwnerFlag'].astype(int)
person_in['NumberCarsOwned'] = person_in['NumberCarsOwned'].astype(int)

# Inspect
person_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   BusinessEntityID      18484 non-null  int64         
 1   EmailPromotion        18484 non-null  int64         
 2   rowguid               18484 non-null  object        
 3   ModifiedDate          18484 non-null  datetime64[ns]
 4   TotalPurchaseYTD      18484 non-null  float64       
 5   DateFirstPurchase     18484 non-null  datetime64[ns]
 6   BirthDate             18484 non-null  datetime64[ns]
 7   MaritalStatus         18484 non-null  category      
 8   YearlyIncome          18484 non-null  category      
 9   Gender                18484 non-null  category      
 10  TotalChildren         18484 non-null  int64         
 11  NumberChildrenAtHome  18484 non-null  int64         
 12  Education             18484 non-null  category      
 13  Occupation      

### Feature development

Some of the exracted information is not very useful in its raw form, but can be used to derive more meaningful features. 

In [47]:
def calculate_age(birthdate, reference_date):
    """
    Calculate age in years from birthdate to a given reference date.

    Args:
    birthdate (pd.Series): Pandas Series of birth dates.
    reference_date (pd.Timestamp): The date from which to calculate the age.

    Returns:
    pd.Series: Age in years as of the reference date.
    """
    return (reference_date - birthdate).dt.days // 365

def days_since_date(first_purchase_date, reference_date):
    """
    Calculate time in days since the first purchase date to a given reference date.

    Args:
    first_purchase_date (pd.Series): Pandas Series of first purchase dates.
    reference_date (pd.Timestamp): The date from which to calculate the years since first purchase.

    Returns:
    pd.Series: Years since first purchase as of the reference date.
    """
    return (reference_date - first_purchase_date).dt.days 


In [48]:
# Set a reference date
reference_date = pd.Timestamp('2024-01-01')

# Apply the functions to calculate age and time in days since first purchase
person_in['Age'] = calculate_age(person_in['BirthDate'], reference_date)
person_in['DaysSinceFirstPurchase'] = days_since_date(person_in['DateFirstPurchase'], reference_date)

# Calulcate years since first purchase
person_in['YearsSinceFirstPurchase'] = person_in['DaysSinceFirstPurchase'] / 365

# Drop datetime columns
person_in = person_in.drop(['BirthDate', 'DateFirstPurchase', 'ModifiedDate'], axis = 1)

# Inspect
person_in

,BusinessEntityID,EmailPromotion,rowguid,TotalPurchaseYTD,MaritalStatus,YearlyIncome,Gender,TotalChildren,NumberChildrenAtHome,Education,Occupation,HomeOwnerFlag,NumberCarsOwned,CommuteDistance,Age,DaysSinceFirstPurchase,YearsSinceFirstPurchase
0,1722,0,6F123B92-FF1E-4F18-A054-F1477D965B36,3499.8504,S,0-25000,F,0,0,Partial High School,Manual,0,2,0-1 Miles,49,7541,20.660274
1,2389,0,0BB7741C-53C5-4502-AABC-AA3E798EC6E7,1085.5000,S,25001-50000,F,5,5,High School,Skilled Manual,1,3,10+ Miles,56,7392,20.252055
2,2391,0,D2535AEE-7780-4CAC-9737-5D23895FD5B5,36.5900,S,greater than 100000,F,0,5,Partial College,Management,1,3,10+ Miles,50,7894,21.627397
3,2397,0,09FB996B-2BD4-454F-A9D2-F0F307D13039,-3.9900,S,25001-50000,F,0,0,Partial High School,Clerical,1,2,5-10 Miles,45,7360,20.164384
4,2401,0,D319CB69-2139-4F0C-AEA6-541B3D5B0412,12.9900,M,25001-50000,F,2,0,Bachelors,Management,1,2,5-10 Miles,84,7139,19.558904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,20737,0,CBD497C2-0C8E-4802-8414-E0C3715CB469,719.5000,M,25001-50000,F,1,1,Bachelors,Skilled Manual,1,1,1-2 Miles,50,7211,19.756164
18480,20751,0,5DD45523-9CB6-49B8-9FB4-80D454711B45,-688.3700,S,greater than 100000,F,0,5,Partial High School,Professional,0,4,10+ Miles,53,7602,20.827397
18481,1725,0,C3D0FAA0-190B-480F-9E38-CBAF6C9C5619,-13.0100,M,50001-75000,M,5,0,Bachelors,Management,1,3,10+ Miles,76,7279,19.942466
18482,1753,0,CF932CF1-F379-4386-A509-28E2EE9086BF,-9.5100,M,50001-75000,M,3,3,Bachelors,Professional,0,2,0-1 Miles,59,7195,19.712329
